This notebook was automatically generated from your MAST-ML run so you can recreate the
plots. Some things are a bit different from the usual way of creating plots - we are
using the [object oriented
interface](https://matplotlib.org/tutorials/introductory/lifecycle.html) instead of
pyplot to create the `fig` and `ax` instances.


In [ ]:
"""
This module contains a collection of functions which make plots (saved as png files) using matplotlib, generated from
some model fits and cross-validation evaluation within a MAST-ML run.

This module also contains a method to create python notebooks containing plotted data and the relevant source code from
this module, to enable the user to make their own modifications to the created plots in a straightforward way (useful for
tweaking plots for a presentation or publication).
"""

import math
import os
import pandas as pd
import itertools
import warnings
import logging
from collections import Iterable
from os.path import join
from collections import OrderedDict

# Ignore the harmless warning about the gelsd driver on mac.
warnings.filterwarnings(action="ignore", module="scipy",
                        message="^internal gelsd")
# Ignore matplotlib deprecation warning (set as all warnings for now)
warnings.filterwarnings(action="ignore")

import numpy as np
from sklearn.metrics import confusion_matrix, roc_curve, auc, precision_recall_curve

import matplotlib
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure, figaspect
from matplotlib.animation import FuncAnimation
from matplotlib.font_manager import FontProperties
import matplotlib.mlab as mlab
from scipy.stats import gaussian_kde
from mpl_toolkits.axes_grid1.inset_locator import mark_inset
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes

# Needed imports for ipynb_maker
from mastml.utils import nice_range
from mastml.metrics import nice_names

import inspect
import textwrap
from pandas import DataFrame, Series

import nbformat

from functools import wraps

matplotlib.rc('font', size=18, family='sans-serif') # set all font to bigger
matplotlib.rc('figure', autolayout=True) # turn on autolayout

# adding dpi as a constant global so it can be changed later
DPI = 250

log = logging.getLogger() # only used inside ipynb_maker I guess



In [ ]:
def stat_to_string(name, value):
    """
    Method that converts a metric object into a string for displaying on a plot

    Args:

        name: (str), long name of a stat metric or quantity

        value: (float), value of the metric or quantity

    Return:

        (str), a string of the metric name, adjusted to look nicer for inclusion on a plot

    """

    " Stringifies the name value pair for display within a plot "
    if name in nice_names:
        name = nice_names[name]
    else:
        name = name.replace('_', ' ')

    # has a name only
    if not value:
        return name
    # has a mean and std
    if isinstance(value, tuple):
        mean, std = value
        return f'{name}:' + '\n\t' + f'{mean:.3f}' + r'$\pm$' + f'{std:.3f}'
    # has a name and value only
    if isinstance(value, int) or (isinstance(value, float) and value%1 == 0):
        return f'{name}: {int(value)}'
    if isinstance(value, float):
        return f'{name}: {value:.3f}'
    return f'{name}: {value}' # probably a string


def plot_stats(fig, stats, x_align=0.65, y_align=0.90, font_dict=dict(), fontsize=14):
    """
    Method that prints stats onto the plot. Goes off screen if they are too long or too many in number.

    Args:

        fig: (matplotlib figure object), a matplotlib figure object

        stats: (dict), dict of statistics to be included with a plot

        x_align: (float), float denoting x position of where to align display of stats on a plot

        y_align: (float), float denoting y position of where to align display of stats on a plot

        font_dict: (dict), dict of matplotlib font options to alter display of stats on plot

        fontsize: (int), the fontsize of stats to display on plot

    Returns:

        None

    """

    stat_str = '\n'.join(stat_to_string(name, value)
                           for name,value in stats.items())

    fig.text(x_align, y_align, stat_str,
             verticalalignment='top', wrap=True, fontdict=font_dict, fontproperties=FontProperties(size=fontsize))


def make_fig_ax(aspect_ratio=0.5, x_align=0.65, left=0.10):
    """
    Method to make matplotlib figure and axes objects. Using Object Oriented interface from https://matplotlib.org/gallery/api/agg_oo_sgskip.html

    Args:

        aspect_ratio: (float), aspect ratio for figure and axes creation

        x_align: (float), x position to draw edge of figure. Needed so can display stats alongside plot

        left: (float), the leftmost position to draw edge of figure

    Returns:

        fig: (matplotlib fig object), a matplotlib figure object with the specified aspect ratio

        ax: (matplotlib ax object), a matplotlib axes object with the specified aspect ratio

    """
    # Set image aspect ratio:
    w, h = figaspect(aspect_ratio)
    fig = Figure(figsize=(w,h))
    FigureCanvas(fig)

    # Set custom positioning, see this guide for more details:
    # https://python4astronomers.github.io/plotting/advanced.html
    #left   = 0.10
    bottom = 0.15
    right  = 0.01
    top    = 0.05
    width = x_align - left - right
    height = 1 - bottom - top
    ax = fig.add_axes((left, bottom, width, height), frameon=True)
    fig.set_tight_layout(False)
    
    return fig, ax


In [ ]:
def plot_residuals_histogram(y_true, y_pred, savepath,
                             stats, title='residuals histogram', label='residuals'):
    """
    Method to calculate and plot the histogram of residuals from regression model

    Args:

        y_true: (numpy array), array of true y data values

        y_pred: (numpy array), array of predicted y data values

        savepath: (str), path to save the plotted precision-recall curve

        stats: (dict), dict of training or testing statistics for a particular run

        title: (str), title of residuals histogram

        label: (str), label used for axis labeling

    Returns:

        None

    """

    # make fig and ax, use x_align when placing text so things don't overlap
    x_align = 0.64
    fig, ax = make_fig_ax(x_align=x_align)

    #ax.set_title(title)
    # do the actual plotting
    residuals = y_true - y_pred

    #Output residuals data and stats to spreadsheet
    path = os.path.dirname(savepath)
    pd.DataFrame(residuals).describe().to_csv(os.path.join(path,'residual_statistics.csv'))
    pd.DataFrame(residuals).to_csv(path+'/'+'residuals.csv')

    #Get num_bins using smarter method
    num_bins = get_histogram_bins(y_df=residuals)
    ax.hist(residuals, bins=num_bins, color='b', edgecolor='k')

    # normal text stuff
    ax.set_xlabel('Value of '+label, fontsize=16)
    ax.set_ylabel('Number of occurences', fontsize=16)

    # make y axis ints, because it is discrete
    #ax.yaxis.set_major_locator(MaxNLocator(integer=True))

    plot_stats(fig, stats, x_align=x_align, y_align=0.90)
    plot_stats(fig, pd.DataFrame(residuals).describe().to_dict()[0], x_align=x_align, y_align=0.60)

    fig.savefig(savepath, dpi=DPI, bbox_inches='tight')


In [ ]:
from numpy import array
from collections import OrderedDict
from io import StringIO
y_true = array([325.49788774, 218.80057061, 214.18575236, 182.12903292,
       121.20205853, 114.1942393 , 120.56546405, 156.51523   ,
       110.503751  ,  95.94694389, 214.25291566, 202.29096841,
       200.04746941, 181.65974631,  71.54661553, 130.71079563,
        99.38751557, 188.72532304, 146.05982872, 191.51161513,
       166.69938886, 179.99544416, 142.48339282, 229.72245207,
        54.94979021,  59.67920269, 114.36792625,  70.6664565 ,
        77.04502053,  91.15693652,  88.46084748,  74.58551681,
       165.79365236, 156.94124086, 168.46495631, 121.20443823,
        58.99391702, 109.47390069,   0.        ,  24.184234  ,
       121.57100715, 183.35068747, 390.4260105 ,  98.51156618,
       254.26505048,  86.04966315,  60.35669252,  89.72129861,
        73.92834539,  80.73776665,  89.50296147, 103.80111055,
       134.18069341,   8.216444  ,  71.44563177,  32.0094037 ,
       179.25872313,   0.        ,  49.34328566,   5.90096117,
        36.36319225,  27.31353641,   0.        , 141.66475147,
        64.50496061, 101.79799033, 128.57645296, 261.7248035 ,
        19.46088327,  29.16320729, 130.86141575, 236.69985867,
        88.18349836, 206.15823625, 119.77883703, 151.79744052,
       289.7129778 ,  59.63498938,  70.83577191,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,  94.50620971,  85.96097361,  89.23565059,
        26.82825386,   0.        ,   0.        ,   0.        ,
         7.74558848,  25.17480383,  53.84255407,   0.        ,
         0.        ,  40.83543394,  43.48823119,  36.50751242,
        43.12521542,   0.        ,  19.2047597 , 161.61585578,
       108.97985761,  14.09444405,  87.36071311,   0.        ,
         0.        ,   0.        ,   0.        ,  11.93148749,
         0.        ,  16.20435918,   8.97062587,  35.50340569,
         7.37896699,  88.38029969,  74.7468692 , 117.12577837,
        62.94422008,  37.34883385, 122.03873942,  33.20006813,
         0.        ,  57.49950031,  61.21370263,  49.15732575,
        28.2257425 ,   0.        ,  86.88965402, 137.40871252,
        53.54285953,  80.45910798,  65.29481037,  58.6508558 ,
        85.23364294, 126.88631207, 468.0346095 , 138.89285472,
        21.17208738,   0.        , 122.31135567,   0.        ,
       130.59739217, 166.18742887, 285.94816437, 106.70233503,
       298.36883132, 207.51571353,  39.9617942 ,  47.36934588,
       197.24625694, 175.56939781, 321.98957475, 367.6244316 ,
       211.47682381, 113.90148661,  48.04117053,  47.45866309,
        35.07688959,  64.97491856,  28.05755061,   9.106651  ,
         0.        ,  10.51108275,   0.        ,   0.        ,
        21.5174731 ,   0.        ,  58.88531967,  72.01080223,
        64.51928629,  66.64638295,  33.27336669,  39.69329769,
        45.2879705 , 169.71976527, 224.40216532, 138.75817334,
       122.98024015, 124.72986509, 152.42942927,  94.65388494,
        83.02690466,   0.        ,   0.        , 141.37372586,
        43.12872844,  25.08521475,  36.35679348,  51.75595534,
        47.24337609, 110.71694286,  23.6410205 ,  44.01037784,
        54.82773069,  57.60884947,  95.49667394,  49.42529029,
        19.80860463,  17.71877028, 125.21536969,  32.611025  ,
        21.12324531,   0.        ,   0.        ,   0.        ,
         5.20671817,   0.        , 129.60153223, 159.11770491,
       191.11304901, 217.25282394, 209.0198447 , 361.57977723,
       106.514784  , 134.67311657,  81.57468903, 129.60777958,
       136.29376713, 111.92047078, 244.14891128, 189.82462955,
       124.31412396, 235.40922492, 192.28465626, 220.16151104,
       196.40913358, 122.67765683, 141.5352297 , 133.95472581,
        87.04599469, 129.34261745, 210.33572768, 103.16515278,
       319.11663064, 292.53608379, 351.81750188, 367.1832279 ,
       237.06234447, 362.46105447, 294.31041312, 177.34615626,
       323.82263694, 232.69981876, 199.00234553, 143.51720548,
       130.55899831, 210.10783532, 212.75663082, 252.16135397,
       184.51653892, 103.45613548,  98.39446077, 187.88525488,
       192.71275543, 197.30900025, 116.27871809, 154.70761089,
       364.70710132, 245.48897712, 338.95548037, 186.71778892,
       230.9176424 , 346.08790771, 142.82619229,  84.93486971,
       144.15449831, 182.22098097, 213.36565687,  64.02417181,
       139.23135132,  54.1587672 ,  61.57636317, 126.81817022,
        45.29321414,  69.20850033,  83.15072959, 184.90772539,
       199.57809553, 233.38366567, 193.51782743, 283.04242932,
       184.42082504, 132.21158822, 204.55489376, 845.33730031,
       950.22790588, 209.325439  , 120.15829999,  67.71649743,
        46.57820663,  82.82809965,  92.79832923, 130.56858186,
       106.91311671, 125.31262584,  36.00668628,  18.90387551,
        12.23389363,  33.14817606,  42.88927876,  29.54783309,
        48.8429076 ,  77.59338173, 161.56318378, 234.93787541,
       147.33684207, 102.11231532, 124.01781744,  95.20604545,
       173.38559503, 174.90906864, 197.17633245, 188.27143603,
       198.01350063, 146.55208232, 182.10184863, 172.75454393,
       146.5521539 , 111.38413582, 181.38757478, 220.40870164,
       189.99865166,  54.91523627,   0.        ,  30.77112483,
       123.93596706, 159.40354586, 114.75434014,  86.69975106,
        97.91858002, 137.58344095, 171.49036331, 208.03565815,
       111.06672169, 117.14316894,  98.78441841, 145.31046867,
       335.32129343, 163.49712402,  89.81673775,  87.6081324 ,
        56.51132963,  66.99094092,  25.29305299,  20.06194164,
        95.30504358,  37.32259412,  95.04900638,  72.94226156,
       113.17048098,  95.6649845 ,  50.44715644,  71.47961918,
        49.79361192,  28.44172814,  35.04593958, 105.85969748,
       143.07547478, 140.73746488,  52.87341873,  87.08840286,
       101.02067384, 129.16762848,  35.07038698,  95.53083323,
         0.        ,  62.09475103,  98.25644159,   0.        ,
       112.1749515 , 124.75938063, 198.10543586, 136.25298986,
        62.32476733,  56.31928935, 160.33463254, 121.97489618,
        74.35889627,  51.91571301,  21.35392387,   9.31924847,
        93.06800775,  21.53240718,  12.40117081,  11.01778613,
         0.        ,  14.2008055 ,  34.10036925,   0.        ,
        49.2018258 ,  29.90345073,   0.        ,  58.14981868,
       139.06215854,  75.6574945 ,  76.84371871,  82.90154417,
       221.76800698,  40.70021975,  52.2740325 ,  70.45416536,
        70.59869671,  65.52025943,  97.23119343,  39.11088786,
         0.        ,  46.95075044,  56.66564979,  87.26704216,
       123.23318414,  37.60933861,   0.        ,  34.86837846,
        80.17227524,  37.42219894,  24.51250794,  66.50124863,
        53.68625362, 173.11303813,  33.00480049,   0.        ,
       289.029515  ,   0.        ,   0.        ,  36.55454693,
        19.41019935])
y_pred = array([0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 1.18364042e+001,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 6.11556778e+001, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       9.60085617e-199, 0.00000000e+000, 0.00000000e+000, 5.88989346e-199,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 3.71719563e-199, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 2.33035113e-316, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 4.49083689e+001, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000, 0.00000000e+000, 0.00000000e+000, 0.00000000e+000,
       0.00000000e+000])
savepath = '/home/nerve/Desktop/skunkworks/lane_schultz/hyper_parameters/test/DoNothing/DoNothing/KernelRidge_DoNothing_DoNothing_GridSearch/RepeatedKFold/split_4/test_residuals_histogram.png'
stats = OrderedDict([('R2', 7.77312500483962e-05), ('root_mean_squared_error', 148.97167704541616), ('mean_absolute_error', 109.11780534719529), ('rmse_over_stdev', 1.5877331116226103)])
title = 'test_residuals_histogram'
label = 'Energy above convex hull (meV/atom)'

In [ ]:
import pandas as pd
from IPython.display import Image, display

plot_residuals_histogram(y_true, y_pred, savepath, stats, title, label)
display(Image(filename='test_residuals_histogram.png'))
